In [1]:
import os
import csv
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import argparse
import ntpath
import sys
import tensorflow as tf
import numpy as np
import datetime
from prettytable import PrettyTable
import pandas as pd

from preprocessing import is_number

In [2]:
path = '/home/wanli/data/Extended_ctr/citeulike_t_extended/paper_info.csv'
null_token = 'NaN'
now = datetime.datetime.now()
    
clean_file_path = path +'.cleaned'


labels = ['doc_id','citeulike_id', 'type', 'journal', 'booktitle', 'series', 'pages', 'year', 'month', 'address']
labels_dtype = {'doc_id':np.int32 , 'citeulike_id':np.int32, 'type': str, 'journal': str, 'booktitle': str, 'series': str,
                  'pages':np.int32, 'month': str, 'address': str}
# Month converter
months = ['apr','aug', 'dec' ,'feb', 'jan' ,'jul' ,'jun' ,'mar' ,'may', 'nov', 'oct', 'sep']
month_convert_func = lambda x: x if x in months else null_token

number_convert_func = lambda x: x if is_number(x) else -1


convert_func= {'month': month_convert_func, 'pages': number_convert_func, 'doc_id': number_convert_func,
               'citeulike_id': number_convert_func}
df = pd.read_table(clean_file_path, delimiter='\t', index_col = 'doc_id', usecols=labels,dtype=labels_dtype,
                     converters=convert_func)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: ParserWarning: Both a converter and dtype were specified for column doc_id - only the converter will be used
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: ParserWarning: Both a converter and dtype were specified for column citeulike_id - only the converter will be used
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: ParserWarning: Both a converter and dtype were specified for column pages - only the converter will be used
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: ParserWarning: Both a converter and dtype were specified for column month - only the converter will be used


In [3]:
print(df.shape)
# count of unique values:
print (df.apply(pd.Series.nunique))

(25976, 9)
citeulike_id    21017
type               12
journal          4109
booktitle        2641
series            430
pages             162
year               85
month              13
address          1659
dtype: int64


In [3]:
#Filter values with frequency less than min_freq
def filter(df, tofilter_list, min_freq):
    for col in tofilter_list:
        to_keep =  df[col].value_counts().reset_index(name="count").query("count > %d" %min_freq)["index"]
        to_keep = to_keep.values.tolist()
        df[col] = [x if x in to_keep else 'NaN' for x in df[col] ]
    return df

tofilter_list = ['journal','booktitle','address']
df = filter(df,tofilter_list,2)

# Convert catigorical feature into one-hot encoding
def dummmy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=True)
        df = df.drop(x,1)
        df = pd.concat([df,dummies], axis=1)
    return df

todummy_list = ['type', 'journal', 'booktitle', 'series', 'month', 'address']
df = dummmy_df(df,todummy_list)
# print(df_dummy.head(2))
# print (df2)
print('----------------------------')
# print (df_dummy['journal'].value_counts())
# .reset_index(name="count").query("count > 3")["index"]
print('----------------------------')
print(df_dummy.shape)
print (df_dummy.apply(pd.Series.nunique))

----------------------------
----------------------------


NameError: name 'df_dummy' is not defined

In [20]:
print(df_dummy.shape)
df_dummy.dropna(inplace=True)
print(df_dummy.shape)
print (df_dummy.apply(pd.Series.nunique))

(25976, 1471)


(21257, 1471)


citeulike_id                                                                                                 21016
pages                                                                                                          162
year                                                                                                            85
type_article                                                                                                     2
type_book                                                                                                        2
type_electronic                                                                                                  2
type_inbook                                                                                                      2
type_incollection                                                                                                2
type_inproceedings                                                              

In [4]:
# min-max normalization:
from sklearn import preprocessing

# x = df.loc[:,todummy_list].values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df.loc[:,todummy_list] = pandas.DataFrame(x_scaled)

values = df.values
imputer = preprocessing.Imputer()
transformed_values = imputer.fit_transform(values)
# count the number of NaN values in each column
print(np.isnan(transformed_values).sum())

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(transformed_values)
transformed_df= pd.DataFrame(x_scaled)

0


In [5]:
print(transformed_df.shape)
print(x_scaled.shape)
# print(transformed_df.nunique())
# print (transformed_df.head(2))

(25976, 1893)
(25976, 1893)


In [13]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# training= np.random.randint(2, size=(x_scaled.shape[0],x_scaled.shape[1]+1))
# training[:,:-1] = x_scaled

y= np.random.randint(2, size=(x_scaled.shape[1]))
# y= y.reshape((y.shape[0],1))
print(x_scaled.shape)
print(y.shape)

# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, np.transpose(np.matrix(x_scaled)), np.transpose(y), cv=kfold, scoring='accuracy')
print(result.mean())

(25976, 1893)
(1893,)


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.491811938722
